In [9]:
# Imports
from datetime import datetime as dt
import pandas as pd

In [2]:
# Reading data from source and creating df
storm_df = pd.read_csv('data/events-US-1980-2021.csv',
                       header=1,
                       parse_dates=['Begin Date', 'End Date'])

In [3]:
storm_df.dtypes

Name                                                     object
Disaster                                                 object
Begin Date                                       datetime64[ns]
End Date                                         datetime64[ns]
Total CPI-Adjusted Cost (Millions of Dollars)           float64
Deaths                                                    int64
dtype: object

In [4]:
storm_df['Disaster'].value_counts()

Disaster
Severe Storm        152
Tropical Cyclone     57
Flooding             36
Drought              29
Winter Storm         20
Wildfire             20
Freeze                9
Name: count, dtype: int64

In [43]:
# Filtering the dataset to match the original visualisation
severe_storm_df = storm_df.loc[storm_df['Disaster'] == 'Severe Storm'].copy(deep=True)
severe_storm_df.reset_index(drop=True, inplace=True)

In [44]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,Total CPI-Adjusted Cost (Millions of Dollars),Deaths
0,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20
1,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33
2,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30
3,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80
4,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1
...,...,...,...,...,...,...
147,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0
148,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0
149,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2
150,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93


In [ ]:
# TODO add seasons columns (2 types), month and year columns 

In [45]:
# created columns for day, month and year to assist in future work
# Went with start date as it would be most relevant for investigating, but can change/add to if needed
day_list = [date.day for date in severe_storm_df['Begin Date']]
month_list = [date.month for date in severe_storm_df['Begin Date']]
year_list = [date.year for date in severe_storm_df['Begin Date']]
severe_storm_df['day_start'], severe_storm_df['month_start'], severe_storm_df['year_start'] = day_list, month_list, year_list

In [46]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,Total CPI-Adjusted Cost (Millions of Dollars),Deaths,day_start,month_start,year_start
0,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20,5,5,1981
1,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33,2,4,1982
2,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30,31,5,1982
3,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80,27,3,1984
4,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1,13,6,1984
...,...,...,...,...,...,...,...,...,...
147,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0,24,6,2021
148,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0,8,7,2021
149,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2,10,8,2021
150,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93,10,12,2021


In [55]:
# meteorologic seasons are easier to define as they encompass entire months
# can also just do season mapping from the beginning, but like to keep it this way to show different methods and for simplifying steps
met_season_numbers = [month % 12 // 3 + 1 for month in severe_storm_df['month_start']]

season_mapping = {1: 'winter', 2: 'spring', 3: 'summer', 4: 'fall'}

met_season = [season_mapping[number] for number in met_season_numbers]
severe_storm_df['met_season'] = met_season

In [56]:
severe_storm_df

,Name,Disaster,Begin Date,End Date,Total CPI-Adjusted Cost (Millions of Dollars),Deaths,day_start,month_start,year_start,met_season,met_season_numbers
0,"Severe Storms, Flash Floods, Hail, Tornadoes (...",Severe Storm,1981-05-05,1981-05-10,1268.6,20,5,5,1981,spring,2
1,Midwest/Plains/Southeast Tornadoes (April 1982),Severe Storm,1982-04-02,1982-04-04,1444.8,33,2,4,1982,spring,2
2,Severe Storms (June 1982),Severe Storm,1982-05-31,1982-06-10,1420.6,30,31,5,1982,spring,2
3,"Tornadoes, Severe Storms, Floods (Spring 1984)",Severe Storm,1984-03-27,1984-04-07,1670.5,80,27,3,1984,spring,2
4,Severe Storms and Hail (June 1984),Severe Storm,1984-06-13,1984-06-17,1222.6,1,13,6,1984,summer,3
...,...,...,...,...,...,...,...,...,...,...,...
147,Central Severe Storms (June 2021),Severe Storm,2021-06-24,2021-06-26,1303.0,0,24,6,2021,summer,3
148,Central Severe Storms (July 2021),Severe Storm,2021-07-08,2021-07-11,1123.2,0,8,7,2021,summer,3
149,North Central Severe Weather (August 2021),Severe Storm,2021-08-10,2021-08-13,1335.3,2,10,8,2021,summer,3
150,"Southeast, Central Tornado Outbreak (December ...",Severe Storm,2021-12-10,2021-12-10,3993.3,93,10,12,2021,winter,1


In [67]:
# Pandas(Index=151, Name='Midwest Derecho and Tornado Outbreak (December 2021)', Disaster='Severe Storm', _3=Timestamp('2021-12-15 00:00:00'), _4=Timestamp('2021-12-15 00:00:00'), _5=1817.6, Deaths=1, day_start=15, month_start=12, year_start=2021, met_season='winter', met_season_numbers=1)

for row in severe_storm_df.itertuples():
    print(row)

Pandas(Index=0, Name='Severe Storms, Flash Floods, Hail, Tornadoes (May 1981)', Disaster='Severe Storm', _3=Timestamp('1981-05-05 00:00:00'), _4=Timestamp('1981-05-10 00:00:00'), _5=1268.6, Deaths=20, day_start=5, month_start=5, year_start=1981, met_season='spring', met_season_numbers=2)
Pandas(Index=1, Name='Midwest/Plains/Southeast Tornadoes (April 1982)', Disaster='Severe Storm', _3=Timestamp('1982-04-02 00:00:00'), _4=Timestamp('1982-04-04 00:00:00'), _5=1444.8, Deaths=33, day_start=2, month_start=4, year_start=1982, met_season='spring', met_season_numbers=2)
Pandas(Index=2, Name='Severe Storms (June 1982)', Disaster='Severe Storm', _3=Timestamp('1982-05-31 00:00:00'), _4=Timestamp('1982-06-10 00:00:00'), _5=1420.6, Deaths=30, day_start=31, month_start=5, year_start=1982, met_season='spring', met_season_numbers=2)
Pandas(Index=3, Name='Tornadoes, Severe Storms, Floods (Spring 1984)', Disaster='Severe Storm', _3=Timestamp('1984-03-27 00:00:00'), _4=Timestamp('1984-04-07 00:00:00'), 

In [61]:
for i in range(1,13):
    print(i % 12 // 3 + 1)

1
1
2
2
2
3
3
3
4
4
4
1


In [70]:
455000/(50*25)


364.0